In [ ]:
import pandas as pd
import pickle as pkl

In [ ]:
df = pd.read_csv('data/memmap/df_memmap.csv')

In [ ]:
all_features = np.load('data/cols_static.npy')

with open('data/task_mapping.pkl', 'rb') as f:
    task_mapping = pkl.load(f)

In [ ]:
lbls_diags = task_mapping['diags']['lbls']
lbls_det = task_mapping['det']['lbls']
col_diags = task_mapping['diags']['col']
col_det = task_mapping['det']['col']

In [ ]:
for c in [cols_diags, cols_det]:
    df[c]=df[c].apply(lambda x:eval(x))

In [ ]:
# note: column 'data' is the index of the waveform from MIMIC-IV-ECG

In [ ]:
# All features (e.g. Multimodal ECG waveform + tabular):

x_train = df[df['strat_fold'].isin(range(0, 18))].drop(columns=['study_id','strat_fold']).reset_index(drop=True)
x_val = df[df['strat_fold'].isin([18])].drop(columns=['study_id','strat_fold']).reset_index(drop=True)
x_test = df[df['strat_fold'].isin([19])].drop(columns=['study_id','strat_fold']).reset_index(drop=True)


x_train = x_train[all_features]

indexes_val = x_val[x_val['ecg_no_within_stay']==0].index
indexes_test = x_test[x_test['ecg_no_within_stay']==0].index

x_val = x_val.iloc[indexes_val][all_features]
x_test = x_test.iloc[indexes_test][all_features]


y_train_diagnoses = df_features[df_features['strat_fold'].isin(range(0, 18))][col_diags].reset_index(drop=True)
y_val_diagnoses = df_features[df_features['strat_fold'].isin([18])][col_diags].reset_index(drop=True)
y_test_diagnoses = df_features[df_features['strat_fold'].isin([19])][col_diags].reset_index(drop=True)

y_val_diagnoses = y_val_diagnoses.iloc[indexes_val]
y_test_diagnoses = y_test_diagnoses.iloc[indexes_test]

y_train_deterioration = df_features[df_features['strat_fold'].isin(range(0, 18))][col_det].reset_index(drop=True)
y_val_deterioration = df_features[df_features['strat_fold'].isin([18])][col_det].reset_index(drop=True)
y_test_deterioration = df_features[df_features['strat_fold'].isin([19])][col_det].reset_index(drop=True)

y_val_deterioration = y_val_deterioration.iloc[indexes_val]
y_test_deterioration = y_test_deterioration.iloc[indexes_test]

In [ ]:
# you can always bring back the missigness, e.g. use xgboost

features_real = all_features[:470]
features_mask = all_features[470:]

mask_array = x_train[features_mask].astype(bool)
x_train_real_with_missing = np.where(mask_array, x_train[features_real], np.nan)